In [1]:

!pip install -U scikit-learn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import os
import pandas as pd
import sklearn as sk



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


2023-08-04 18:50:11.341529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.config.list_logical_devices()

2023-08-04 18:50:13.230883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 18:50:13.235460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 18:50:13.235669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 18:50:13.237718: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-04 18:50:13.237895: I tensorflow/compile

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
x = pd.read_csv("Data/data.csv")

In [4]:
x.describe()

,text,label
count,65101,65698
unique,59237,2
top,Highlights: The Trump presidency on April 13 a...,False
freq,8,33894


In [5]:
x.groupby('label').describe()

text                                                               
       count unique                                                top freq
label                                                                      
False  33297  27653  ELEMENTARY SCHOOL PLANS ‘BLACKS ONLY’ FIELD TR...    4
True   31804  31584  Highlights: The Trump presidency on April 13 a...    8

In [6]:
#drop non unique from x
x = x.drop_duplicates()

In [7]:
x_FALSE = x[x['label'] == False]
x_TRUE = x[x['label'] == True]


In [8]:
#make it so X_False, and X_True are the same size
x_TRUE = x_TRUE[:len(x_FALSE)]

In [9]:
x_FALSE.shape

(27654, 2)

In [10]:
x_TRUE.shape

(27654, 2)

In [11]:
x_balanced = pd.concat([x_TRUE, x_FALSE], axis=0)

In [12]:
x_balanced.dtypes

text     object
label      bool
dtype: object

In [13]:
x_balanced.sample(10)

,text,label
25124,Australian women go pro on back of investment ...,True
2949,Conservative ‘Christians’ Get Their Asses Han...,False
15096,Insane! Huge Financial Disaster On The Horizon...,False
52373,Episode #160 – SUNDAY WIRE: ‘Hail to the Deplo...,False
7947,This Conservative Wants To Let Scalia Vote Fr...,False
8564,Michigan Government To EPA: ‘You Can’t Tell U...,False
41530,Iran names nuclear negotiating team member jai...,True
45799,Internet Flasher,False
36217,"Erdogan, Putin to discuss Syria, Jerusalem dur...",True
27761,Trump objects to terminology of border adjustm...,True


In [14]:
#convert the text to string,
x_balanced["text"] = x_balanced["text"].astype(str)

In [15]:
x_balanced['label'] = x_balanced['label'].apply(lambda x: 1 if x == False else 0)

In [16]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_balanced["text"], x_balanced["label"], test_size=0.2, random_state=42, stratify=x_balanced["label"])

In [17]:
X_train.head()

31600    House committee to hold hearing on Wells Fargo...
33532    U.S. weakens retirement advice rule, respondin...
31319    Trump tied with Clinton in Utah after lewd rem...
36206    Questions on free movement, red tape linger in...
37235    Greek government under fire over would-be Saud...
Name: text, dtype: object

In [18]:
#import tensorflow hub
!pip install tensorflow-hub
!pip install tensorflow-text
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text
import tensorflow_text as text


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [19]:
#grab the bert model
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [20]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

2023-08-04 18:50:27.814667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84369534, -0.51368064, -0.88893497, ..., -0.748008  ,
        -0.7533265 ,  0.91974676],
       [-0.8720002 , -0.50530386, -0.9443905 , ..., -0.8583416 ,
        -0.7173763 ,  0.8808192 ]], dtype=float32)>

In [21]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.99108]], dtype=float32)

In [23]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

In [25]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5


2023-08-04 18:50:34.586320: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f10ac00b0f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-04 18:50:34.586358: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-08-04 18:50:34.590322: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-04 18:50:34.708370: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600


   1/1383 [..............................] - ETA: 2:12:01 - loss: 0.7394 - accuracy: 0.5312 - precision: 0.6000 - recall: 0.6316

2023-08-04 18:50:34.789471: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1383/1383 [==============================] - 345s 246ms/step - loss: 0.4267 - accuracy: 0.8081 - precision: 0.8084 - recall: 0.8078
Epoch 2/5
1383/1383 [==============================] - 339s 245ms/step - loss: 0.3496 - accuracy: 0.8485 - precision: 0.8446 - recall: 0.8542
Epoch 3/5
1383/1383 [==============================] - 339s 245ms/step - loss: 0.3350 - accuracy: 0.8561 - precision: 0.8530 - recall: 0.8606
Epoch 4/5
1383/1383 [==============================] - 339s 245ms/step - loss: 0.3324 - accuracy: 0.8575 - precision: 0.8549 - recall: 0.8613
Epoch 5/5
1228/1383 [=========================>....] - ETA: 38s - loss: 0.3278 - accuracy: 0.8589 - precision: 0.8574 - recall: 0.8620

In [ ]:
model.save('VerifyAI_DO03_RD.h5')

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()